# [Rinna-3.6B で LangChain を試す](https://note.com/npaka/n/n872f37dd36cd)
- 参照元の実行環境はColabだけどVSCode上で実行します。
- Python 3.11.1
- RTX3070Ti
- ひとまず普通に動く

In [1]:
# (1) パッケージのインストール。

# パッケージのインストール
!pip install transformers sentencepiece accelerate langchain bitsandbytes
!pip install xformers
!pip install scipy
!pip install torchinfo

In [2]:
# (2) トークナイザーとモデルの準備。

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# modelname = "rinna/bilingual-gpt-neox-4b-instruction-sft"
# modelname = "rinna/japanese-gpt-neox-3.6b-instruction-ppo"
modelname = "rinna/japanese-gpt-neox-3.6b-instruction-sft-v2"
# modelname = "rinna/japanese-gpt2-small"
# modelname = "rinna/japanese-gpt2-xsmall"

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    modelname,
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    modelname,
    # torch_dtype=torch.bfloat16, #GPU
    # torch_dtype=torch.bfloat32, #CPU
    # device_map="auto",
)

/home/mhiavio/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
# from torchinfo import summary

# summary(
#     model,
#     input_size=(batch_size, 3, 224, 224),
#     col_names=["output_size", "num_params"],
# )

In [3]:
# (3) パイプラインの準備。

from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# パイプラインの準備
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    temperature=0.1,
)
llm = HuggingFacePipeline(pipeline=pipe)

In [9]:
# (4) プロンプトテンプレートとLLMChainの準備。

from langchain import PromptTemplate, LLMChain

# プロンプトテンプレートの準備
template = """ユーザー: 次の文章を読んで質問に答えてください。
文章: 範馬勇次郎は、「地上最強の生物」「オーガ（鬼）」「巨凶」など数々の称号を持つ。「地上最強」の異名の通り、何者をも超える絶対的な力をほしいままにしている。しかし、物語が進むにつれ力無き者の希望となる偉人を尊敬していたり、高級店でのマナーなども熟知していたりするなど、様々な側面を見せるようになる。
質問: {question}
システム:"""
template = template.replace("\n", "<NL>")
prompt = PromptTemplate(template=template, input_variables=["question"])

# LLMChainの準備
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
# (5) 質問応答。

# 質問応答
# question = "後藤ひとりのバンド名は何ですか？"
question = "勇次郎の運転マナーはどうですか？"
print(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


 勇次郎は、運転マナーが良く、安全運転を心がけています。
